## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-05-10-52-41 +0000,bbc,Trump hails 'historic' peace deal between DR C...,https://www.bbc.com/news/articles/cjrjn88jqn4o...
1,2025-12-05-10-51-28 +0000,nyt,Air Traffic Control Issue Briefly Grounds Flig...,https://www.nytimes.com/2025/12/05/world/europ...
2,2025-12-05-10-48-00 +0000,wsj,Eurozone Economy Posts Stronger Growth on Inve...,https://www.wsj.com/economy/global/eurozone-ec...
3,2025-12-05-10-47-02 +0000,nyt,Trump’s Relationship With the President of FIFA,https://www.nytimes.com/video/world/1000000105...
4,2025-12-05-10-40-00 +0000,wsj,Chinese Forces Battle Dizzying Altitudes to Ex...,https://www.wsj.com/world/china/chinese-forces...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,48
503,new,20
840,he,12
124,was,12
379,gaza,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
250,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...,122
162,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...,114
225,2025-12-04-21-11-38 +0000,nypost,Trump boasts heads of Rwanda and Congo will be...,https://nypost.com/2025/12/04/us-news/trump-bo...,90
226,2025-12-04-21-10-58 +0000,nypost,Venezuelan dictator Maduro says his phone call...,https://nypost.com/2025/12/04/world-news/madur...,88
133,2025-12-05-00-21-26 +0000,nypost,Trump sees peace ‘eventually’ in Ukraine after...,https://nypost.com/2025/12/04/world-news/trump...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
250,122,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...
132,81,2025-12-05-00-22-00 +0000,wsj,"In a briefing for lawmakers, Navy Adm. Frank “...",https://www.wsj.com/politics/national-security...
50,50,2025-12-05-06-12-19 +0000,bbc,Putin says Russia will take Donbas by force or...,https://www.bbc.com/news/articles/cgexgwr1wrro...
295,48,2025-12-04-16-32-33 +0000,nyt,"Yasser Abu Shabab, Militant Leader Backed by I...",https://www.nytimes.com/2025/12/04/world/middl...
170,48,2025-12-04-22-57-09 +0000,latimes,Halle Berry declares Gavin Newsom 'should not ...,https://www.latimes.com/entertainment-arts/sto...
162,46,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...
36,42,2025-12-05-08-46-10 +0000,nypost,US and Kenya sign first of what are expected t...,https://nypost.com/2025/12/05/world-news/us-an...
38,41,2025-12-05-08-27-31 +0000,wsj,Stock Market Today: Nasdaq Futures Lead Gains ...,https://www.wsj.com/livecoverage/stock-market-...
155,40,2025-12-04-23-42-38 +0000,latimes,"Supreme Court rules for Texas Republicans, all...",https://www.latimes.com/politics/story/2025-12...
51,38,2025-12-05-06-12-02 +0000,nypost,19-year-old arrested for role in shocking mob ...,https://nypost.com/2025/12/05/us-news/19-year-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
